In [ ]:
import sys
sys.path.append('../../PlateTectonicTools/')

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import colors as mplcolors
import pygplates
import pygmt

from gprm import ReconstructionModel
from gprm.datasets import Rocks, Reconstructions, Paleogeography, Geology
from gprm.utils.raster import to_anchor_plate

import sys
#sys.path.append('/Users/simon/OneDrive/Andes_works//python/')
sys.path.append('../../andes_paleoelevation//python/')
import joyful_geochemistry as joy
import joyful_mapping as joymap

import collections

%matplotlib inline
%load_ext autoreload
%autoreload 2


############## Settings for Scotese Paleomap
PaleomapDictionary = {}
PaleomapDictionary['name'] = 'Paleomap'
PaleomapDictionary['reconstruction_model'] = Reconstructions.fetch_Scotese()
PaleomapDictionary['raster_sequence'] = Paleogeography.fetch_Paleomap(resolution='06m')
PaleomapDictionary['maximum_time'] = 540.
PaleomapDictionary['time_bin_size'] = 20.
PaleomapDictionary['anchor_plate_id'] = 0
PaleomapDictionary['raster_anchor_plate_id'] = 0

########## Geochemistry Inputs
#df = joy.geochem_from_csv('../datafiles/geochem_merge_20221026.csv')
#df = joy.geochem_from_csv('../datafiles/geochem_global_20230124.csv',
#                          longitude_field_name='longitude', latitude_field_name='latitude')

model_dir = '../../andes_paleoelevation//luffi/REM_surfaces_csv/'
gc_interpolator_dict = joy.make_gc_interpolator_dict(model_dir)



In [ ]:
df = pd.read_csv('../datafiles/geochem_global_20230124_subduction_M2021.csv')
df = gpd.GeoDataFrame(df, geometry=gpd.GeoSeries.from_wkt(df['geometry']), crs='EPSG:4326')

'''
df = joymap.select_orogens(df,gdf=None, 
                           orogen_names=[#'Cordilleran', 
                                         #'Terra Australis', 
                                         #'Solonker', 
                                         'Indosinian', 
                                         #'Variscan-Hercynian',
                                         #'Scotian',
                                         #'Uralian',
                                         #'Mongol',
                                         #'Kazakh',
                                         #'Caledonia',
                                         #'Alpine-Himalayan'
                           ], 
                           #continent_names='South America',
                           #region=[-100, -50, -60, 20]
                          )
'''

sz_distance_limit = 750.

df = df.query('distance_to_sz<@sz_distance_limit').reset_index(drop=True)

df.plot()

In [ ]:
#### TODO
# filter out samples that fall outside polygon age ranges


df = PaleomapDictionary['reconstruction_model'].assign_plate_ids(df, copy_valid_times=True)
df = df.query('AnalysisAge<=FROMAGE')

## To File
calibration = 'luffi'
mohometer_selection = 42
df_filt = joy.filter_the_database(df, filter_method=calibration, age_min=0.1)
df_filt['FROMAGE'] = df_filt['AnalysisAge']+5.
df_filt['TOAGE'] = df_filt['AnalysisAge']-5.
#df.to_file('../datafiles/geochem_global_20230124_subduction_M2021_750km.shp')

#df['FROMAGE'].hist(bins=50)



time_bin_size = 10
space_bin_size = 2

binned_df_cat = []
for reconstruction_time in np.arange(0,540,10):
    
    df_filt_r = joymap.geochem_timeslice(df_filt, reconstruction_time, time_bin_size, calibration=calibration,
                                         reconstruction_model=None)
    
    if df_filt_r is not None:
        elevation_estimates = joy.get_elevations(df_filt_r, 
                                                gc_interpolator_dict=gc_interpolator_dict,
                                                calibration=calibration,
                                                mohometer_selection=mohometer_selection)
        
        binned_df = joy.bin_elevations(df_filt_r.geometry, elevation_estimates, space_bin_size)
        
        binned_df['FROMAGE'] = reconstruction_time+time_bin_size/2
        binned_df['TOAGE'] = reconstruction_time-time_bin_size/2
        
        binned_df = gpd.GeoDataFrame(binned_df, geometry=gpd.points_from_xy(binned_df.x, binned_df.y), crs=4326)
        
        binned_df_cat.append(binned_df)
        #break

        
gpd.pd.concat(binned_df_cat).to_file('../datafiles/elevation_estimates_2deg_10Myr.shp')



In [ ]:
###########################################
MODEL = PaleomapDictionary
###########################################

region = [-180, 180, -89, 89]
#region = [-76, -66, -25, -12]
projection="Q20c"
perspective = [210, 35]

column_marker_size = '0.2c'
###########################################



time_bin_size = 10. #MODEL['time_bin_size']
space_bin_size = 5.

#calibration = 'Hu'
#mohometer_selection = ['la_yb_elevation']

calibration = 'luffi'
mohometer_selection = 42

anchor_plate_id = MODEL['anchor_plate_id']
raster_anchor_plate_id = MODEL['raster_anchor_plate_id']

reconstruction_model = MODEL['reconstruction_model']
raster_sequence = MODEL['raster_sequence']


label_x = 0.35
label_y = 0.96
colorbar_position='JBL+jBL+o14c/1.5c+w8c/0.8c+h'


if isinstance(mohometer_selection, list):
    mohometer_description_string = '|'.join(mohometer_selection)
else:
    mohometer_description_string = str(mohometer_selection)

    
    
for reconstruction_time in np.arange(0,MODEL['maximum_time']+time_bin_size,time_bin_size):
    
    fig = pygmt.Figure()

    binned_elevations = joymap.timeslice_plot(df, reconstruction_time,
                                              time_bin_size, space_bin_size, 
                                              fig, reconstruction_model, raster_sequence,  
                                              anchor_plate_id=anchor_plate_id, raster_anchor_plate_id=raster_anchor_plate_id,
                                              gc_interpolator_dict=gc_interpolator_dict, 
                                              calibration=calibration, mohometer_selection=mohometer_selection,
                                              region=region, projection=projection, perspective=perspective, 
                                              column_marker_size=column_marker_size, coastlines=False,
                                              volcanics=None, return_type='binned_elevations')
    
    joymap.add_labels(fig, reconstruction_time, x=label_x, y=label_y,
                      add_colorbar=True, colorbar_position=colorbar_position)
    
    fig.savefig(
        '../images/sequence_{:s}/elevations_{:s}_{:s}_{:0.0f}_{:0.0f}Ma.png'.format(MODEL['name'],
                                                                                    calibration,
                                                                                    mohometer_description_string,
                                                                                    space_bin_size,
                                                                                    reconstruction_time))
    #fig.show(width=1000)
    
    fig = pygmt.Figure()

    binned_residuals = joymap.timeslice_plot(df, reconstruction_time,
                                             time_bin_size, space_bin_size, 
                                             fig, reconstruction_model, raster_sequence,
                                             anchor_plate_id=anchor_plate_id, raster_anchor_plate_id=raster_anchor_plate_id,
                                             gc_interpolator_dict=gc_interpolator_dict, 
                                             calibration=calibration, mohometer_selection=mohometer_selection,
                                             region=region, projection=projection, perspective=perspective, 
                                             column_marker_size=column_marker_size, coastlines=False,
                                             volcanics=None, residuals=True, return_type='binned_residuals')
    
    joymap.add_labels(fig, reconstruction_time, x=label_x, y=label_y,
                      add_colorbar=True, colorbar_position=colorbar_position,
                      colorbar_title='Residual Elevation [m]')
    
    fig.savefig(
        '../images/sequence_{:s}/elevation_residuals_{:s}_{:s}_{:0.0f}_{:0.0f}Ma.png'.format(MODEL['name'],
                                                                                             calibration,
                                                                                             mohometer_description_string, 
                                                                                             space_bin_size,
                                                                                             reconstruction_time))
    #fig.show(width=1000)
    
    joymap.residuals_crossplot(
        binned_elevations, 
        binned_residuals,
        fname='../images/sequence_{:s}/crossplot_{:s}_{:s}_{:0.0f}_{:0.1f}Ma.png'.format(MODEL['name'],
                                                                                         calibration,
                                                                                         mohometer_description_string, 
                                                                                         space_bin_size,
                                                                                         reconstruction_time))
    #break
    
